### <center> Lab 5-1: UDS over CAN
## <center> ENGR 580A2: Secure Vehicle and Industrial Networking
## <center><img src="https://www.engr.colostate.edu/~jdaily/Systems-EN-CSU-1-C357.svg" width="600" />
### <center> Instructor: Dr. Jeremy Daily<br>Fall 2020

# Prerequisites
We'll need a file that logs traffic from a unified diagnostic service (UDS) session. UDS is used by many companies, and is especially popular among european companies. For our example, we'll use a DDEC Reports session.

DDEC Reports is a freely available fleet management tool for Detroit Diesel Engines. If you wanted to use it, you'd need an RP1210 compliant vehicle diagnostics adapter (VDA), like the DPA5. A google search for "DDEC Reports Download" usually points you in the right direction. 


We've already gathered a DDEC Reports file and the candump log file for when it was made. We need to download that first. If you are running this notebook on a BeagleBone, make sure it is connected to the Internet. 

In [31]:
# Install the requests library so we can download files from the Internet
# uncomment the following line if requests is not installed
#%pip install requests --user
import requests

In [32]:
# Download the candump log file from Dr. Daily's website
url = r'https://www.engr.colostate.edu/~jdaily/J1939/files/candump_DDECReportsExtaction032819.txt'
r = requests.get(url)
candump_file = r.text
print(candump_file[:203])

(1531228460.025007) can1 18FDD001#FFFFFF3030FFFFFF
(1531228460.025565) can1 14FEF131#CFFFFFF300FFFF30
(1531228460.026155) can1 18FEDB01#E900E900E900FFFF
(1531228460.026721) can1 18FD9401#4C01FFFFFFFFF


In [35]:
# Download the DDEC Reports file from Dr. Daily's website
url = r'https://www.engr.colostate.edu/~jdaily/J1939/files/DDECReportsExtaction032819.pdf'
r = requests.get(url)
# Write the received binary pdf data to a file:
with open('DDECReportsExtaction032819.pdf','wb') as pdf_file:
    pdf_file.write(r.content)

If the block above successfully ran, then you should be able to open the PDF file with this link [DDECReportsExtaction032819.pdf](DDECReportsExtaction032819.pdf)

In [36]:
# Download the DDEC Reports file from Dr. Daily's website
url = r'https://www.engr.colostate.edu/~jdaily/J1939/files/DDECReportsExtaction032819.XTR'
r = requests.get(url)
# Write the received binary DDEC Reports data to a file:
with open('DDECReportsExtaction032819.XTR','wb') as xtr_file:
    xtr_file.write(r.content)

# ISO 14229 or ISO 15765
These ISO standards define a method of communication that is primarily focused on diagnosic services. 

For SAE J1939 networks, there is a special parameter group number 55808 (0xDA00) that is used for all UDS traffic. This means we can filter out all the other traffic and only focus on the UDS messages.

Source address 241 (0xF1) is the world-wide harmonized (WWH) off-board diagnositics (OBD) Tester for OBD tester as defined in WWH-OBD ISO 15765-4.


In [55]:
# Extract UDS messages from the CAN log
uds_messages = []
for line in candump_file.split('\n'):
    elements = line.replace("#"," ").split()
    try:
        if "18DA" in elements[2]:
            can_id = int(elements[2],16) 
            sa = can_id & 0xFF
            da = (can_id & 0xFF00) >> 8
            data = bytes.fromhex(elements[3])
            timestamp = float(elements[0].strip('()'))
            uds_messages.append({'timestamp':timestamp,'sa':sa,'da':da,'data':data})
    except IndexError:
        # This is usually an empty string at the end
        print(elements)
uds_messages[:5]

[]


[{'timestamp': 1531228470.611006,
  'sa': 241,
  'da': 0,
  'data': b'\x03"\xf1\x00\xff\xff\xff\xff'},
 {'timestamp': 1531228470.631073,
  'sa': 0,
  'da': 241,
  'data': b'\x07b\xf1\x00\x02\x01\x0e\x01'},
 {'timestamp': 1531228472.603126,
  'sa': 241,
  'da': 0,
  'data': b'\x02\x10\x03\x00\x00\x00\x00\x00'},
 {'timestamp': 1531228472.621486,
  'sa': 0,
  'da': 241,
  'data': b'\x06P\x03\x00\x14\x00\xc8\x01'},
 {'timestamp': 1531228473.115446,
  'sa': 241,
  'da': 0,
  'data': b'\x03"\xf1\x00\x00\x00\x00\x00'}]

With only the ISO 15765 data in the list, we can begin to understand some of the diagnostic communications. Let's take this one line at a time. For reference, we can use the ISO specification (which is expensive) or this poster:

https://automotive.softing.com/fileadmin/sof-files/pdf/de/ae/poster/UDS_Faltposter_softing2016.pdf

Since all of the UDS information is in the data field, we only derive source and destination address from the CAN ID. The first nibble of the first byte in the data frame is one of 3 things:
1. If the first nibble is a 0, then it is a single frame message. The second nibble is the message size.
2. If the first nibble is a 1, then this is the first frame of a long message. The next 12 bits represent the message size.
3. If the first nibble is a 2, then this is a consecutive frame from the first message.
4. If the first nibble is a 3, then it is a session control message. 

https://en.wikipedia.org/wiki/ISO_15765-2

Since 12 bits are allocated to describe the message size, the total amount of data that can be transfered in 1 session is 4096 bytes, which is over twice the size of the maximum for the J1939 Transport Protocol.

### Single Frame Message: Read Data By Identifier

In [62]:
# Let's look at the first UDS message:
uds_messages[0]

{'timestamp': 1531228470.611006,
 'sa': 241,
 'da': 0,
 'data': b'\x03"\xf1\x00\xff\xff\xff\xff'}

This message comes from source address 241, which is the 

In [61]:
# We can pretty-print the data:
" ".join(["{:02X}".format(b) for b in uds_messages[0]['data']])

'03 22 F1 00 FF FF FF FF'

The first nibble is a zero, so this is a single frame message. 

The next nibble is 3, which means there are 3 bytes to this message, which are 0x22, 0xF1, 0x00

The first byte in this sequence is called the *service identifier* (SID). The upper right table in the poster shows the different service ids.

![UDS_Service_Overview.png](UDS_Service_Overview.png)

Based on this message the SID is 0x22, which is the **Read Data By Identifier** service name (RDBI). It does not have a sub function, so the identifier to read is 0xF100.

When we look up the 0xF100 identifier in Appendix C.1 of the ISO 14229-3 standard, it says that this parameter is Manufacturer Specific, which means we may not get any other information about the data that comes next. 

In [65]:
# Look at the next record
print(uds_messages[1])

{'timestamp': 1531228470.631073, 'sa': 0, 'da': 241, 'data': b'\x07b\xf1\x00\x02\x01\x0e\x01'}


In [66]:
# This is the response to
" ".join(["{:02X}".format(b) for b in uds_messages[1]['data']])

'07 62 F1 00 02 01 0E 01'

This response from the engine controller (SA = 0) is a single frame with seven bytes: 0x07.

The response service identifier is indicated by adding 0x40 to the requested SID. This is the same as setting the the 6th bit: 0b01000000


In [67]:
sid = 0x22
sid_response = sid | 0x40 #set the repsonse bit
print("response SID: 0x{:02X}".format(sid_response))

response SID: 0x62


The parameter for the response is still 0xF100 and the data for the response is 0x20010E01. We don't know what this parameter from the engine means, but presumably the diagnostic software application knows what to do.  

In [68]:
# let's look at the next sequence of messages
for msg in uds_messages[2:10]:
    print(msg)

{'timestamp': 1531228472.603126, 'sa': 241, 'da': 0, 'data': b'\x02\x10\x03\x00\x00\x00\x00\x00'}
{'timestamp': 1531228472.621486, 'sa': 0, 'da': 241, 'data': b'\x06P\x03\x00\x14\x00\xc8\x01'}
{'timestamp': 1531228473.115446, 'sa': 241, 'da': 0, 'data': b'\x03"\xf1\x00\x00\x00\x00\x00'}
{'timestamp': 1531228473.131437, 'sa': 0, 'da': 241, 'data': b'\x07b\xf1\x00\x02\x01\x0e\x03'}
{'timestamp': 1531228473.828286, 'sa': 241, 'da': 0, 'data': b"\x02'\x05\x00\x00\x00\x00\x00"}
{'timestamp': 1531228473.841246, 'sa': 0, 'da': 241, 'data': b'\x04g\x05\x11\x8e\x01\x0e\x03'}
{'timestamp': 1531228474.095685, 'sa': 241, 'da': 0, 'data': b"\x04'\x06\xc0,\x00\x00\x00"}
{'timestamp': 1531228474.111126, 'sa': 0, 'da': 241, 'data': b'\x02g\x06\x11\x8e\x01\x0e\x03'}


### Session Control Message
The next message is a single frame, 2-byte message from the OBD Tester device (the diagnostic tool).

In [71]:
print(uds_messages[2])

{'timestamp': 1531228472.603126, 'sa': 241, 'da': 0, 'data': b'\x02\x10\x03\x00\x00\x00\x00\x00'}


In [72]:
" ".join(["{:02X}".format(b) for b in uds_messages[2]['data']])

'02 10 03 00 00 00 00 00'

This message SID is 0x10, which is the Diagnostic Session Control (DSC). It has a subfunction, which is indicated by the following byte, 0x03. The highest two bits of the value are both zero. According to the Sub-Function Byte of UDS, this means the suppressPosRspMsgIndicatioBit is not set, so the ECU is expected to respond.

When looking up SID of 0x10, we find the subfunction of this SID is the diagnostic Session Type. A value of 0x30 means 
```
extendedDiagnosticSession (EXTDS)
This diagnosticSession can be used to enable all diagnostic services required to support the adjustment of functions like "Idle Speed, CO Value, etc." in the server's memory. It can also be used to enable diagnostic services, which are not specifically tied to the adjustment of functions.
```
according to the ISO-14229-3 standard.

So we are looking at an extended diagnostic session.


In [73]:
# Look at the response:
print(uds_messages[3])
" ".join(["{:02X}".format(b) for b in uds_messages[3]['data']])

{'timestamp': 1531228472.621486, 'sa': 0, 'da': 241, 'data': b'\x06P\x03\x00\x14\x00\xc8\x01'}


'06 50 03 00 14 00 C8 01'

The response from the engine controller to the OBD tool is a single frame with 6 bytes (0x06). The response SID is has 0x40 added to the origination SID: 0x50 = 0x10 + 0x40.

The diagnostic Function Type is still 0x03, which is an extended session. This takes up 2 bytes, there are still four to go.

According to ISO 14229, Tables 26-28, we can interpet the next four bytes as the sessionParameterRecord, which contain ECU timing requirements. These bytes have the following definitions:
```
P2Server_max (high byte)
P2Server_max (low byte)
P2*Server_max (high byte)
P2*Server_max (low byte)
```
These parameters have the following definitions:

| Parameter | Description | # of bytes | Resolution | minimum value | maximum value |
| :-- | :-- | --- | --- | --- | --- |
| `P2Server_max` |Default P2Server_max timing supported by the server for the activated diagnostic session. | 2 | 1 ms | 0 ms | 65,535 ms |
| `P2*Server_max` | Enhanced (NRC 0x78) P2Server_max supported by the server for the activated diagnostic session. | 2 | 10 ms | 0 ms | 655,350 ms |

Enhanced Negative Response Code (NRC) 0x78 is `requestCorrectlyReceived-ResponsePending` (RCRRP)

```This NRC indicates that the request message was received correctly, and that all parameters in the request message were valid, but the action to be performed is not yet completed and the server is not yet ready to receive another request. As soon as the requested service has been completed, the server shall send a positive response message or negative response message with a response code different from this.```

In [77]:
P2Server_max = struct.unpack('>H',uds_messages[3]['data'][3:5])[0]
print("The ECU can support up to {} milliseconds for P2Server_max".format(P2Server_max))

The ECU can support up to 20 milliseconds for P2Server_max


In [78]:
P2_star_Server_max = struct.unpack('>H',uds_messages[3]['data'][5:7])[0]
print("The ECU can support up to {} milliseconds for P2*Server_max".format(P2_star_Server_max*10))

The ECU can support up to 2000 milliseconds for P2*Server_max


## Security Session Setup

In [96]:
# look at the transactions
for i in range(6,10):
    print("From: {:3d}, To: {:3d}, ".format(uds_messages[i]['sa'],uds_messages[i]['da']),
          " ".join(["{:02X}".format(b) for b in uds_messages[i]['data']]))

From: 241, To:   0,  02 27 05 00 00 00 00 00
From:   0, To: 241,  04 67 05 11 8E 01 0E 03
From: 241, To:   0,  04 27 06 C0 2C 00 00 00
From:   0, To: 241,  02 67 06 11 8E 01 0E 03


The first message is from the OBD Tester to the engine controller of a single frame, 2 byte message. The SID is 0x27, which is **Security Access**. The Sub-Function is 0x05, which is **requestSeed**. The tool is asking the ECU for a seed so it can authenticate itself to perform diagnostics. 

The second message is the response from the engine with only 4 bytes. The Response code is 0x67 = 0x27 + 0x40 and the positive response message is the **securitySeed**. This is a 16-bit number with the high byte first and low byte second (big endian, motorolla format).

The third message is also Security Access, but the Sub-Function is 0x60 which is **sendKey**. The key value are the next two bytes and they are calculated by the service tool based on the securitySeed value. In other words, there is a challenge-response function that maps 0x118E to 0xC02C:
$$ f(0x118E) = 0xC02C $$
From a cybersecurity stand point, this so-called security is insufficient to stop anything but an accidental session on the bus. 16-bits are far too few to be cryptographically sound. 

**THIS APPROACH IS NOT ACTUALLY SECURE!**

We could figure out $f$. How?

Finally, the last message is from the Engine with a positive response to the key exchange. Notice there are only 2 bytes in this response, 0x02. This means the bytes 11 8E 01 0E 03 have no meaning and you can see they were left over from the last transmission from the engine. They can be ignored. 

Now the security setup has been satisfied, and the session can continue.

##  Multiple Frame Message
The next block of messages make use of the ISO transport protocol. The first nibble in the messages are greater than zero. This is byte is called the protocol control information (PCI). 

In [99]:
# look at the next 3 messages
for i in range(10,13):
    print("From: {:3d}, To: {:3d}, ".format(uds_messages[i]['sa'],uds_messages[i]['da']),
          " ".join(["{:02X}".format(b) for b in uds_messages[i]['data']]))

From: 241, To:   0,  10 0D 2E F1 5C 00 00 00
From:   0, To: 241,  30 08 14 11 8E 01 0E 03
From: 241, To:   0,  21 13 03 1C 00 00 00 00


The first message is from the OBD tool to the engine. The first nibble is a 1, which means it is the first frame in a multi frame message. The next 3 nibbles (12 bits) are the length of the message in bytes.

In [100]:
# Compute the message length
message_bytes = struct.unpack(">H",uds_messages[10]['data'][0:2])[0]
print(message_bytes)

4109


In [101]:
# Since we only want 3 nibbles, we have to mask off the initial 1:
message_bytes &= 0x0FFF
print(message_bytes)

13


In [102]:
0xD

13

So, we are expecting 13 bytes of data. The data count starts with the bytes immediatly following the count. There are 6 of the 13 bytes on the first line: `2E F1 5C 00 00 00`

The second line is from the engine to the tool and is a 0x30 Flow control PCI message. The value of 3 in the first nibble indicates it is a flow control message. The second nibble, 0, indicates it's clear to send. The options are 0 = Clear To Send, 1 = Wait, 2 = Overflow/abort. The next byte, 0x80, indicates the number of frames that may be sent before waiting for the next flow control frame. A value of zero allows the remaining frames to be sent without flow control or delay. The third byte, 0x14, is the Separation Time (ST) or the minimum time in milliseconds between frames. In this case, frames should be send 20 ms apart. 

In [105]:
0x14

20

In the last message, the remaining 7 of the 13 bytes are transmitted. The PCI nibble is 2 and the next nibble is 1, which means it is the first of the consecutive frames. The second nibble will increment as subsequent consecutive frames arrive. The last of the message is `13 03 1C 00 00 00 00`

In [112]:
full_message = bytes.fromhex('2E F1 5C 00 00 00 13 03 1C 00 00 00 00')
print('Message length:', len(full_message))
print(" ".join(["{:02X}".format(b) for b in full_message]))

Message length: 13
2E F1 5C 00 00 00 13 03 1C 00 00 00 00


This starts with a SID of 0x2E, which is **Write Data By Identififer**. The data identifier is 0xF15C and the data to write is 0x00 00 00 13 03 1C 00 00 00 00. The data identifier falls in the identificationOptionVehicleManufacturerSpecificDataIdentifier range.

In [114]:
for i in range(13,18):
    print("From: {:3d}, To: {:3d}, ".format(uds_messages[i]['sa'],uds_messages[i]['da']),
          " ".join(["{:02X}".format(b) for b in uds_messages[i]['data']]))

From:   0, To: 241,  03 6E F1 5C 8E 01 0E 03
From: 241, To:   0,  05 31 01 03 04 01 00 00
From:   0, To: 241,  04 71 01 03 04 01 0E 03
From: 241, To:   0,  03 22 F1 95 00 00 00 00
From:   0, To: 241,  10 16 62 F1 95 00 02 56
